## Bring in dependencies

In this code cell we'll bring in all the dependencies we'll need for later use.

Go ahead and run the following code cell. **There should be no ouput.**

In [ ]:

from langchain.document_loaders.pdf import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# WML python SDK
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM



## Some important variables

In this next code cell you'll define some variables that will be used in order to interact with your instance of watsonx.ai.

Go ahead and run the following code cell. **There should be no ouput**

In [ ]:
# Update the global variables that will be used for authentication in another function

# Set your project id here
watsonx_project_id = ""

# API key here
api_key = ""

# Set your region here: us-south, us-east, eu-de, etc.
REGION = "us-south"

url = "https://" + REGION + ".ml.cloud.ibm.com"


## Understanding the code

In this next code cell we'll create some functions that we can use later to interact easier with watsonx.ai. These functions are ***get_model()***, ***get_lang_chain_model()***, and ***answer_question_from_doc()***:

- ***get_model()***: creates a model object that will be used to invoke the LLM. Since the ***get_model()*** function is parametrized, it's the same in all examples.
- ***get_lang_chain_model()***: creates a model wrapper that will be used with the _LangChain_ API.
- ***answer_question_from_doc()*** specifies model parameters, loads the PDF file, creates an index from the loaded document, the instantiates and invokes the chain.

Go ahead and run the following code cell. **There should be no ouput**.

In [ ]:
def get_model(model_type,max_tokens,min_tokens,decoding,temperature):

    generate_params = {
        GenParams.MAX_NEW_TOKENS: max_tokens,
        GenParams.MIN_NEW_TOKENS: min_tokens,
        GenParams.DECODING_METHOD: decoding,
        GenParams.TEMPERATURE: temperature
    }

    model = Model(
        model_id=model_type,
        params=generate_params,
        credentials={
            "apikey": api_key,
            "url": url
        },
        project_id=watsonx_project_id
    )

    return model

def get_lang_chain_model(model_type,max_tokens,min_tokens,decoding,temperature):

    base_model = get_model(model_type,max_tokens,min_tokens,decoding,temperature)
    langchain_model = WatsonxLLM(model=base_model)

    return langchain_model


## Gluing it together

The next function, `answer_questions_from_doc`, that we create is created to help combine the previous three that we defined. This is the wrapper that we will call when we want to interact with watsonx.ai.

Go ahead and run the following code cell. **There should be no ouput**.

In [ ]:
def answer_questions_from_doc(file_path, question):

  # Specify model parameters
  model_type = "meta-llama/llama-2-70b-chat"
  max_tokens = 300
  min_tokens = 100
  decoding = DecodingMethods.GREEDY
  temperature = 0.7

  # Get the watsonx model that can be used with LangChain
  model = get_lang_chain_model(model_type, max_tokens, min_tokens, decoding, temperature)

  loaders = [PyPDFLoader(file_path)]

  index = VectorstoreIndexCreator(
      embedding=HuggingFaceEmbeddings(),
      text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)).from_loaders(loaders)

  chain = RetrievalQA.from_chain_type(llm=model,
                                      chain_type="stuff",
                                      retriever=index.vectorstore.as_retriever(),
                                      input_key="question")

  # Invoke the chain
  response_text = chain.invoke(question)

  # print model response
  print("--------------------------------- Generated response -----------------------------------")
  print(response_text["result"])
  print("*********************************************************************************************")


## Answering some questions

The next code cell will use all the previous code we've created so far to source information from the input documents and ask a question about them using watsonx.ai (Notice the use of the `answer_questions_from_doc`).

To do so we'll pass in a question we want to ask, the PDF file we want to reference for said question, and finally the name of the collection where the embeddings of the file exist.

Notice the commented questions as well? Feel free to uncomment these or create some or your own to ask

Go ahead and run the next code cell. **You _will_ see output from this cell**

In [ ]:
# Test answering questions based on the provided .pdf file
question = "What is Generative AI?"
# question = "What does it take to build a generative AI model?"
# question = "What are the limitations of generative AI models?"
file_path = "https://raw.githubusercontent.com/CloudPak-Outcomes/Outcomes-Projects/main/L4assets/watsonx.ai-Assets/Documents/Generative_AI_Overview.pdf"

answer_questions_from_doc(file_path, question)
